In [ ]:
from pylab import *
%matplotlib inline

In [ ]:
def f(x, omega =1.):
    return sin(x*omega)

def g(h_de_x,x, args = []):
    if len([args]) == 0:
        return exp(-x)*h_de_x(x)
    else:
        return exp(-x)*h_de_x(x,args)

In [ ]:
x = linspace(0,2*pi,200)
for i in range(5):
    subplot(121)
    plot(x,f(x,i))
    r = f(x,i)
    subplot(122)
    plot(x,g(f,x,i))
show()

In [ ]:
def int_rk4(ec,p_ini,x, args = 0,h=0.001):
    if abs(x[1]-x[0])<=5.*h:
        h = abs(x[1]-x[0])/100
    tiempos = arange(x[0],x[1]+h,h)
    sol = zeros((len(tiempos),len(p_ini)))
    sol[0,:] = p_ini
    
    if args == 0:
        for i in xrange(len(tiempos)-1):
            k1 = ec(sol[i,:],tiempos[i])
            k2 = ec(sol[i,:]+0.5*h*k1,tiempos[i]+0.5*h)
            k3 = ec(sol[i,:]+0.5*h*k2,tiempos[i]+0.5*h)
            k4 = ec(sol[i,:]+h*k3,tiempos[i]+h)
            sol[i+1,:] = sol[i,:] + (h/6.0)*(k1+2*k2+2*k3+k4)
        return tiempos,sol
    else:
        for i in xrange(len(tiempos)-1):
            k1 = ec(sol[i,:],tiempos[i], args)
            k2 = ec(sol[i,:]+0.5*h*k1,tiempos[i]+0.5*h, args)
            k3 = ec(sol[i,:]+0.5*h*k2,tiempos[i]+0.5*h, args)
            k4 = ec(sol[i,:]+h*k3,tiempos[i]+h, args)
            sol[i+1,:] = sol[i,:] + (h/6.0)*(k1+2*k2+2*k3+k4)
        return tiempos,sol

Consideramos ahora el caso de un tiro parabólico con resistencia al aire. La resistencia por la fricción provoca una aceleración que esta dada por: $$\dot{\vec{u}}=-\frac{g}{m}\hat{j}-\frac{f}{m}\frac{\vec{u}}{||\vec{u}||}$$

In [ ]:
def ec_mov(r_v, t, fric = 0., g = 9.81, m = 1.):
    u = sqrt(r_v[2]**2 + r_v[3]**2)
    dx = r_v[2]
    dy = r_v[3]
    if fric == 0.:
        dvx = 0.
        dvy = -g/m
    else:
        dvx = -fric([r_v[2], r_v[3]])[0]
        dvy = -g/m -fric([r_v[2], r_v[3]])[1]
    return array([dx,dy,dvx,dvy])
def fricNul(v):
    return array([0,0])
def fricV1(v, gamma = 0.1):
    f_x = gamma*v[0]
    f_y = gamma*v[1]
    return array([f_x,f_y])
def fricV2(v, gamma = 0.1):
    norma = sqrt(v[0]**2+v[1]**2)
    f_x = gamma*norma*v[0]
    f_y = gamma*norma*v[1]
    return array([f_x,f_y])

In [ ]:
def movPlaneta(x,t):
    r = sqrt(x[0]**2 + x[1]**2)
    drx = x[2]
    dry = x[3]
    dvx = -x[0]/r**3
    dvy = -x[1]/r**3
    return array([drx,dry,dvx,dvy])

In [ ]:
p = [0,0,5,5]
t,sol_id = int_rk4(ec_mov, p, [0,1])
plot(sol_id[:,0], sol_id[:,1],label="ideal")
t,sol_fr = int_rk4(ec_mov, p, [0,1], fricV1)
plot(sol_fr[:,0], sol_fr[:,1], label = "lineal")
t,sol_fr = int_rk4(ec_mov, p, [0,1], fricV2)
plot(sol_fr[:,0], sol_fr[:,1], label = "cuadratico")
title('Tiro con Resistencia')
legend()
show()

Ahora deseamos hallar el punto donde la solución cruza el eje x (y=0) para hacer un rebote del proyectil. 

In [ ]:
inicio = [0,0,5,5]
N=150
trayectoria=zeros((N,len(inicio)))
mi_dt=0.1
trayectoria[0,:]=array(inicio)

for i in range(N-1):
    t,sol=int_rk4(ec_mov,trayectoria[i,:],[0,mi_dt],fricV1)
    
    if sol[-1,:][1]<0:
        print "buscando punto de rebote"
        dt_busqueda = mi_dt/2
        y_izq = trayectoria[i,:]
        t,sol = int_rk4(ec_mov,trayectoria[i,:],[0,dt_busqueda],fricV1)
        y_med = sol[-1,:]
        
        while abs(y_izq[1]) > 1e-5:
            if y_izq[1]*y_med[1] < 0.0:
                reinicio = y_izq
            else:
                reinicio = y_med
            dt_busqueda = dt_busqueda/2
            t,sol = int_rk4(ec_mov,reinicio,[0,dt_busqueda],fricV1)
            y_izq = reinicio
            y_med = sol[-1,:]
            
        print y_izq
        trayectoria[i+1,:] = y_izq
        trayectoria[i+1,:][3] *= -1
    
    else:
        trayectoria[i+1,:] = sol[-1,:]
    
tfv,sol_fv=int_rk4(ec_mov,inicio,[0,1],fricV1)    
plot(sol_fv[:,0],sol_fv[:,1],'o')    
plot(trayectoria[:,0],trayectoria[:,1],'--')
show()

In [ ]:
p = [0,0,5,5]
N = 500
trayectoria = zeros((N,len(p)))
mi_dt = 0.1
trayectoria[0,:] = array(p)
for i in range(N-1):
    t, sol = int_rk4(ec_mov,trayectoria[i,:],[0,mi_dt],fricV1)
    if sol[-1,:][1] < 0:
        dt_busqueda = mi_dt/2
        y_izq = trayectoria[i,:]
        t, sol = int_rk4(ec_mov,trayectoria[i,:],[0,dt_busqueda],fricV1)
        y_med = sol[-1,:]

        while abs(y_izq[1])>1e-5:
            dt_busqueda = dt_busqueda/2
            if y_izq[1]*y_med[1] < 0:
                reinicio = y_izq
            else:
                reinicio = y_med
            t, sol = int_rk4(ec_mov,reinicio,[0,dt_busqueda],fricV1)
            
            y_izq = reinicio
            y_med = sol[-1,:]
            
        trayectoria[i+1,:] = y_izq
        trayectoria[i+1,:][3] *= -1.
    else:
        trayectoria[i+1,:] = sol[-1,:]

plot(trayectoria[:,0],trayectoria[:,1], label="Te odio Diego")
plot(sol_fv[:,0],sol_fv[:,1], label="Por siempre")
legend()
show()